In [ ]:
!pip install scikit-learn pandas emoji tensorflow numpy sentence_transformers
!pip install sklearn_crfsuite
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install fastai==1.0.46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=a33aaf4a9231b3901e605a02621f5a64f41715931ca682c517c8fd1c704f1827
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import unicodedata
import pythainlp
import re
import emoji
import tensorflow as tf
import numpy as np
from sentence_transformers import SentenceTransformer, util
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word)]

    res = ungroup_emoji(res)
    return res

In [ ]:
train_df = pd.read_csv('./train_data_fake_news.csv', encoding = "utf-8")
val_df = pd.read_csv('./valid_data_fake_news.csv', encoding = "utf-8")
test_df = pd.read_csv('./test_data_fake_news.csv', encoding = "utf-8")

In [ ]:
train_df

,text,label
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1
...,...,...
2358,เดี๋ยวนี้มีใบทุเรียนน้ำแกโรคมะเร็งหายขาดจ้า,1
2359,อาหาร8ประเภทที่ไม่ควรนำไปอุ่นซ้ำอีกรอบ|||จะต้อ...,1
2360,เภสัชเผยยากลุ่มNSAIDsห้ามกินร่วมกันเสี่ยงอันตร...,0
2361,สบส.ฟันธง!!วัยทำงานเกือบครึ่งอนาคตเสี่ยงโรคเรื...,0


In [ ]:
train_df_pre = train_df.copy()
val_df_pre = val_df.copy()
test_df_pre = test_df.copy()
train_df_pre['text'] = train_df_pre['text'].map(preprocess_func)
val_df_pre['text'] = val_df_pre['text'].map(preprocess_func)
test_df_pre['text'] = test_df_pre['text'].map(preprocess_func)

In [ ]:
word_count = []
for text in train_df_pre['text']:
    for word in text:
        word_count.append(word)

In [ ]:
VOCAB_SIZE = len(set(word_count))
VOCAB_SIZE

25447

In [ ]:
MAX_SEQ_LEN = 250
EMBEDDING_DIM = 300

In [ ]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(train_df_pre['text'])

In [ ]:
train_seq = tokenizer.texts_to_sequences(train_df_pre['text'])

In [ ]:
train_seq[0]

[1190, 3263, 15506, 40, 24, 4, 41, 83, 902, 540]

In [ ]:
train_pad = pad_sequences(train_seq, maxlen=MAX_SEQ_LEN)

In [ ]:
train_pad

array([[   0,    0,    0, ...,   83,  902,  540],
       [   0,    0,    0, ..., 1064, 2247,  618],
       [  48,   29,  256, ...,  762,   33, 2373],
       ...,
       [  50,   98,   51, ...,  171,   73,   26],
       [   3,  232,    7, ...,    5,   35, 2311],
       [   0,    0,    0, ..., 3563, 3794,   73]], dtype=int32)

In [ ]:
val_seq = tokenizer.texts_to_sequences(val_df_pre['text'])
val_pad = pad_sequences(val_seq, maxlen=MAX_SEQ_LEN)

In [ ]:
print(train_df_pre['text'][0])
print(train_seq[0])

['ไก่', 'ลุง', 'เคน', 'กิน', 'แล้ว', 'มี', 'แต่', 'สาร', 'เร่ง', 'โต']
[1190, 3263, 15506, 40, 24, 4, 41, 83, 902, 540]


In [ ]:
from pythainlp import word_vector
model = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

Corpus: thai2fit_wv
- Downloading: thai2fit_wv 0.1


  0%|          | 0/62452646 [00:00<?, ?it/s]

In [ ]:
thai2dict = {}
for word in model.index2word:
    thai2dict[word] = model[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thai2vec.head(10)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
ที่,0.308956,-0.097699,0.116745,0.215612,0.015768,-0.064163,0.062168,0.039649,0.864940,0.846904,...,-0.142418,0.033241,0.171581,-0.624864,-0.009358,0.449131,0.120130,-0.122195,-0.450617,-0.071318
และ,0.010751,-0.618971,0.129665,0.035460,-0.007560,0.027607,0.397824,0.026543,0.254075,0.168328,...,-0.105786,0.180930,-0.101630,0.070885,-0.037263,0.183606,-0.049088,-0.672288,-1.293044,0.592576
เป็น,-0.015736,-0.258926,0.052953,0.153728,-0.005985,-0.021081,0.041088,0.057312,1.633230,0.442729,...,-0.009408,-0.252576,-0.305512,0.372542,0.049151,0.568470,0.266586,0.400800,-0.784650,0.197369
ของ,-0.189711,-0.174774,0.171124,-0.186771,0.054294,-0.114150,-1.109456,-0.094466,-0.447015,0.042377,...,-0.168676,-0.148738,0.680404,0.097702,0.020270,0.182967,-0.083949,0.006287,-0.707434,-0.070234
มี,-0.156962,-0.231863,0.080312,0.323157,0.215695,0.055145,0.420794,0.016842,0.256759,0.832864,...,-0.044267,-0.147186,-0.105424,0.907078,0.009299,0.550953,0.139337,0.031696,-0.670379,-0.008048
ได้,-0.428813,-0.031194,0.041922,-0.036608,-0.008106,0.076470,-0.782270,0.033361,0.606864,0.440520,...,0.024458,-0.025031,0.103389,-0.078255,0.034323,0.459774,-0.748643,0.337775,-0.487408,-0.511535
"""""""""",-0.287710,0.064193,0.205076,0.146356,-0.071343,-0.039451,-1.845461,0.163763,1.018096,0.272786,...,0.051024,-0.532856,-0.131856,-0.090323,-0.058895,0.151262,-0.420358,0.055971,-0.930814,0.163908
การ,0.239587,-0.303620,0.079953,-0.453045,-0.528826,-0.161692,0.235725,-0.099673,0.691668,0.536159,...,-0.110436,-0.297495,-0.217414,0.045158,0.066647,0.190095,-0.304333,-0.724927,-0.995488,-0.716609
(,-0.120522,-0.355783,0.168180,-0.377733,-0.158624,-0.047249,0.361140,0.161460,0.913314,0.345037,...,0.116285,-0.318218,-0.356664,0.519889,0.130475,0.125772,0.101328,-0.382658,-1.205359,0.340139
),-0.086848,-0.155231,0.133015,-0.039913,0.183761,0.115142,-1.940854,-0.066565,-2.399744,0.146722,...,0.019406,-0.181474,0.099863,0.516092,0.201697,0.249139,0.252957,1.138815,-0.018209,0.232265


In [ ]:
word_index = tokenizer.word_index

In [ ]:
num_word = len(word_index) + 1
embedding_matrix = np.zeros((num_word, EMBEDDING_DIM))
for word, i in word_index.items():
  if i < num_word:
    emb_vec = thai2dict.get(word)
    if emb_vec is not None:
      embedding_matrix[i] = emb_vec

In [ ]:
embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.08955997e-01, -9.76990014e-02,  1.16745003e-01, ...,
        -1.22194998e-01, -4.50616986e-01, -7.13180006e-02],
       [ 1.07509997e-02, -6.18970990e-01,  1.29665002e-01, ...,
        -6.72288001e-01, -1.29304397e+00,  5.92576027e-01],
       ...,
       [-4.32070009e-02, -1.18960002e-02, -4.48869988e-02, ...,
        -4.84299986e-03, -3.74279991e-02, -2.79239994e-02],
       [ 2.86699994e-03, -1.68169998e-02, -8.02349970e-02, ...,
         9.22999985e-04, -2.99740005e-02, -2.05700006e-02],
       [ 2.41999995e-04, -2.09790003e-02, -8.99669975e-02, ...,
         2.56699999e-03, -2.91530006e-02, -2.18809992e-02]])

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.initializers import Constant

model = Sequential()
model.add(layers.Embedding(num_word, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_SEQ_LEN))
model.add(layers.Conv1D(100, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 300)          7634400   
                                                                 
 conv1d (Conv1D)             (None, 248, 100)          90100     
                                                                 
 global_max_pooling1d (Globa  (None, 100)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1010      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,725,521
Trainable params: 7,725,521
Non-trainable params: 0
____________________________________________

In [ ]:
model.fit(train_pad, train_df_pre['label'],
                     epochs=20,
                     verbose=2,
                     validation_data=(val_pad, val_df_pre['label']),
                     batch_size=10)

Epoch 1/20
237/237 - 25s - loss: 0.0178 - accuracy: 0.9962 - val_loss: 0.4061 - val_accuracy: 0.8557 - 25s/epoch - 105ms/step
Epoch 2/20
237/237 - 27s - loss: 0.0143 - accuracy: 0.9966 - val_loss: 0.4279 - val_accuracy: 0.8538 - 27s/epoch - 112ms/step
Epoch 3/20
237/237 - 26s - loss: 0.0125 - accuracy: 0.9979 - val_loss: 0.3530 - val_accuracy: 0.8834 - 26s/epoch - 108ms/step
Epoch 4/20
237/237 - 24s - loss: 0.0106 - accuracy: 0.9979 - val_loss: 0.4611 - val_accuracy: 0.8577 - 24s/epoch - 102ms/step
Epoch 5/20
237/237 - 26s - loss: 0.0093 - accuracy: 0.9983 - val_loss: 0.3945 - val_accuracy: 0.8715 - 26s/epoch - 111ms/step
Epoch 6/20
237/237 - 24s - loss: 0.0084 - accuracy: 0.9979 - val_loss: 0.3893 - val_accuracy: 0.8913 - 24s/epoch - 102ms/step
Epoch 7/20
237/237 - 24s - loss: 0.0123 - accuracy: 0.9979 - val_loss: 0.3640 - val_accuracy: 0.8953 - 24s/epoch - 102ms/step
Epoch 8/20
237/237 - 24s - loss: 0.0034 - accuracy: 0.9992 - val_loss: 0.3694 - val_accuracy: 0.8893 - 24s/epoch - 103

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_df_pre['text'])
test_pad = pad_sequences(test_seq, maxlen=MAX_SEQ_LEN)

In [ ]:
loss, accuracy = model.evaluate(test_pad, test_df_pre['label'])
print("Accuracy: {:.4f}".format(accuracy))

16/16 [==============================] - 1s 32ms/step - loss: 0.4815 - accuracy: 0.8836
Accuracy: 0.8836
